## 2개의 층

케라스 API를 사용해서 패션 MNIST 데이터셋을 불러옵니다.

In [ ]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()

그 다음 이미지의 픽셀값을 0\~255에서 0~1 범위로 변환합니다.  
그리고 28 x 28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼칩니다.  
마지막으로 훈련 세트와 검증 세트로 나눕니다.

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_input.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

이제 인공 신경망 모델에 층을 2개 추가합니다.  
입력층과 출력층만 있었지만, 이번에는 그 사이에 밀집층이 하나 더 추가되는 것입니다.  
이렇게 입력층과 출력층 사이에 있는 모든 층을 **은닉층(hidden layer)**이라고 부릅니다.

은닉층에는 활성화 함수를 적용할 수 있습니다.  
활성화 함수는 신경망 층의 선형 방정식의 계산값에 적용하는 함수입니다. 출력층에 적용했던 소프트맥스 함수도 활성화 함수입니다.  

출력층에 적용하는 활성화 함수는 종류가 제한되어 있습니다.
- 이진 분류: 시그모이드 함수를 사용
- 다중 분류: 소프트맥스 함수를 사용

이에 비해 은닉층의 활성화 함수는 비교적 자유롭습니다.  
대표적으로
- 시그모이드 함수
- 렐루(ReLU)함수 등을 사용

In [ ]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

100개의 뉴런을 가진 밀집층을 dense1으로 만듭니다. sigmoid를 활성화 함수로 지정했고, 케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해줘야 하기 때문에 입력을 합니다.  
dense2는 기존과 같은 출력층입니다.

은닉층의 뉴런 개수를 정하는 데는 특별한 기준이 없습니다.  
몇 개의 뉴런을 두어야 할지 판단하기 위해서는 상당한 경험이 필요합니다.  
그래도 확실한 건 출력층의 뉴런보다는 많게 만들어야 합니다.

## 심층 신경망 만들기

In [ ]:
model = keras.Sequential([dense1, dense2])

Sequential 클래스의 객체를 만들 때 여러 층을 추가하려면 층을 리스트로 만들어 전달해야 합니다.  
주의할 점은, 출력층을 가장 마지막에 두어야 합니다.  
처음에서 마지막 순서대로 층이 생성되기 때문입니다.

케라스 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있습니다.

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층을 만들 때 name 매개변수로 이름을 지정할 수도 있습니다.  
출력 크기를 보면 (None, 100)입니다. 첫번째 차원은 샘플의 개수를 나타냅니다. 샘플의 개수가 아직 정의되어 있지 않기에 None이 나옵니다.  
왜냐하면 케라스 모델의 fit()메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행하기 때문입니다(=> 미니배치 경사 하강법 사용). 케라스의 기본 미니배치 크기는 32개인데, 이 값은 fit(batch_size=)로 바꿀 수 있습니다. 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정합니다.

dense1 파라미터 크기 계산  
입력 픽셀 784 x 은닉층 뉴런 100 + 뉴런마다 절편 100 = 78500

dense2 파라미터 크기 계산  
은닉층 뉴런 100 x 출력층 뉴런 10 + 뉴런마다 절편 10 = 1010

## 층을 추가하는 다른 방법

따로 변수로 저장하지 말고 Sequential 클래스 생성 시, 바로 리스트 안에 넣어서 생성해도 됩니다.  
이번엔 name도 설정해서 해보겠습니다.

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='fassion_mnist_model')

In [ ]:
model.summary()

Model: "fassion_mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


위 방법은 편리하지만, 많은 층을 추가하려면 너무 길어집니다.  
아래와 같이 add() 메서드를 사용하면 깔끔합니다.

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 이 모델을 compile하고 훈련해보겠습니다.

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.8017 - accuracy: 0.7172
Epoch 2/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.6046 - accuracy: 0.7759
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5674 - accuracy: 0.7934
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5471 - accuracy: 0.8039
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5347 - accuracy: 0.8062


추가된 층이 성능을 향상시켰다는 것을 알 수 있습니다.  
인공 신경망에 몇 개의 층을 추가하더라도 compile(), fit() 메서드의 사용법은 동일합니다.

## 렐루 함수
렐루 함수는 이미지 분석에서 높은 성능을 내는 함수입니다.  

초창기 인공 신경망의 은닉층에서는 활성화 함수로 시그모이드 함수를 많이 사용했습니다만, 이 함수는 양쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다는 단점이 있습니다.  
특히 층이 많은 심층 신경망일수록 효과가 누적되어 학습을 더 어렵게 만듭니다.  

개선하기 위해 제안된 함수가 바로 렐루(ReLU) 함수입니다.  
입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 입력을 그냥 통과시키고, 음수일 경우에는 0으로 만듭니다.  

렐루 함수는 심층 신경망에서 뛰어납니다.


### Flatten 층
렐루 함수를 적용하기 전에 케라스에서 제공하는 편리한 층 하나를 더 살펴봅니다.  
패션 NMIST 데이터는 28x28 크기인데 인공 신경망에 주입하기 위해 reshape() 메서드를 사용할 수도 있지만,  
케라스에서 제공하는 Flatten 층을 사용할 수도 있습니다.

입력에 곱해지는 가중치나 절편이 없습니다. 성능에 기여하는 바는 없지만 입력층과 은닉층 사이에 추가하기 때문에 층이라 합니다.

In [ ]:
# Flatten층을 입력층 바로 뒤에 추가
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               78500     
                                                                 
 dense_12 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이전에 있던 코드와 같지만, reshape()를 사용하지 않았습니다.

In [ ]:
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5298 - accuracy: 0.8142
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3944 - accuracy: 0.8560
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3536 - accuracy: 0.8720
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3338 - accuracy: 0.8800
Epoch 5/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3186 - accuracy: 0.8854


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3567 - accuracy: 0.8767


[0.35668954253196716, 0.8766666650772095]

성능이 조금 향상되었습니다.

## 옵티마이저
이전에 보고 지나온 하이퍼파라미터는 다음과 같습니다.
- 은닉층의 개수
- 뉴런 개수: `Dense(10)`
- 활성화 함수: sigmoid, relu 등
- 층의 종류: Dense 등
- 배치 사이즈 매개변수: `fit(batch_size=)`
- 에포크 매개변수: `fit(epochs=)`


추가적으로 조정할 수 있는 것이 있습니다. 다양한 종류의 경사 하강법 알고리즘을 제공하는 것이 있는데 이것을 옵티마이저(optimizer)라 합니다.  
`compile(optimizer=)` default: RMSprop

즉, 옵티마이저함수는 하이퍼파라미터 배치나 조정을 가장 적절하게 바꿔주는 역할을 합니다.  
배치사이즈, 학습률 등을 분석가가 설정한대로 진행하다가 옵티마이저 함수는 학습의 마지막쯤에서 자동으로 배치사이즈나 학습률등을 조정하는 역할을 합니다.

In [ ]:
# model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 위와 같은 코드
# sgd = keras.optimizers.SGD()
# model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

# 학습률을 바꾸고 싶다면
# sgd = keras.optimizers.SGD(learning_rate=0.1)

# 기본값
# keras.optimizers.SGD(learning_rate=0.01, momentum=0, nesterov=False)
# momentum > 0 로 설정하면, 이전의 그레디언트를 가속도처럼 사용하는 '모멘텀 최적화'를 사용.
# nesterov=True로 설정하면, 네스테로프 모멘텀 최적화(네스테로프 가속 경사)를 사용.

In [ ]:
# Adagrad
# adagrad = keras.optimizers.Adagrad() # default: learning_rate=0.001
# model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

# RMSprop
# rmsprop = keras.optimizers.RMSprop() # default: learning_rate=0.001
# model.comile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# 모멘텀 최적화의 장점 + RMSprop의 장점 = Adam
# default: learning_rate=0.001

In [ ]:
# Adam
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5162 - accuracy: 0.8197
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3889 - accuracy: 0.8603
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3487 - accuracy: 0.8728
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3248 - accuracy: 0.8816
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3037 - accuracy: 0.8873


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3294 - accuracy: 0.8820


[0.32935330271720886, 0.8820000290870667]

기본 RMSprop 사용했을 때와 거의 같은 성능이지만 조금 낫습니다.